### how to solve inter hyphen issue https://stackoverflow.com/questions/52293874/why-does-spacy-not-preserve-intra-word-hyphens-during-tokenization-like-stanford

In [1]:
import re
import os
import json
import spacy
import nltk
from spacy.tokenizer import Tokenizer
from spacy.util import compile_prefix_regex, compile_infix_regex, compile_suffix_regex
from benepar.spacy_plugin import BeneparComponent
from spacy.tokens import Doc
nlp = spacy.load('en')
nlp.add_pipe(BeneparComponent('benepar_en2'))

/home/pengfei/miniconda3/envs/benepar/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/pengfei/miniconda3/envs/benepar/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/pengfei/miniconda3/envs/benepar/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/pengfei/miniconda3/envs/benepar

In [2]:
# def custom_tokenizer(nlp):
# #     infix_re = re.compile(r'''[~-]''')
#     infix_re = compile_infix_regex(nlp.Defaults.infixes)
#     prefix_re = compile_prefix_regex(nlp.Defaults.prefixes)
#     suffix_re = compile_suffix_regex(nlp.Defaults.suffixes)

#     return Tokenizer(nlp.vocab, 
#                      prefix_search=prefix_re.search,
#                                 suffix_search=suffix_re.search,
#                                 infix_finditer=infix_re.finditer,
#                                 token_match=None)


def custom_tokenizer(text):
    tokens = nltk.word_tokenize(text)
    # with this:
    return Doc(nlp.vocab, tokens)

nlp.tokenizer = custom_tokenizer

In [3]:
def get_doc(doc_path):
    with open(doc_path) as f:
        txt = f.read().replace('\n', ' ')
    doc = nlp(txt)
    sentences = []
    sent_token_offset = 0
    for i, sent in enumerate(doc.sents):
        if i == 0: continue
        words = []
        dependencies = []
        count = 0
        for word in sent:
            charoffsetbegin = word.idx
            charoffsetend = word.idx + len(word)
            word_pos = word.pos_
            word_string = str(word)
            if str(word) == '\"':
                if count % 2 == 0:
                    word_string = '``'
                else:
                    word_string = '\'\''
            word_combo = [word_string, 
                          {"CharacterOffsetBegin": charoffsetbegin, 
                                      "CharacterOffsetEnd": charoffsetend, 
                                      "Linkers": [], 
                                      "PartOfSpeech": word.pos_}]
            words.append(word_combo)

            dep_r = word.dep_
            parent = str(word.head) + '-' + str(word.head.i - sent_token_offset)
            child = str(word) + '-' + str(word.i - sent_token_offset)
            dep = [dep_r, parent, child]
            dependencies.append(dep)

        parsetree = sent._.parse_string
        parsetree = re.sub(r'\(_SP ((.|\n)*)\)', '', parsetree)
        sentences.append({"parsetree": parsetree, "words": words, "dependencies": dependencies})
        sent_token_offset += len(sent)
    return sentences

In [4]:
folder_path = '/home/pengfei/data/pdtb3-dataset/all/raw/'
files = os.listdir(folder_path)
all_data = {}
files = [f for f in files if f[4:6] in ['00','01','21','22']]

In [ ]:
for f in files:
    doc_path = os.path.join(folder_path, f)
    doc = get_doc(doc_path)
    all_data[f] = {'sentences': doc}

In [ ]:
with open('pdtb-parse.json', 'w') as f:
    json.dump(all_data, f)

In [171]:
# json_path = '/home/pengfei/data/pdtb3-dataset/all/conll/train/pdtb-parses.json'
# with open(json_path,  'r') as f:
#     parse_dict = json.load(f)